In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [2]:
from ta.trend import TRIXIndicator
def Trix(window_para, upper, lower, to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        result.loc[code == result['code'].shift(window_para),'trix'] = TRIXIndicator(close=result['close_weekly'],window=window_para).trix()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['trix']>upper),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['trix']<lower),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, upper, lower

In [8]:
Trix(2,0, -5, to_csv = False)

MDD 0.21091172609279019


(3.1017940961120494, 2)

In [11]:
optimize_result = []
window_para =10
for upper in range(0,50, 5):
    for window_para in range(2,20,2):
            x=Trix(window_para,upper, -5, to_csv = False)
            optimize_result.append(x)    
            print(window_para, upper, lower, x)
optimize_result = pd.DataFrame(optimize_result)

MDD 0.21091172609279019
2 0 -5 (3.1017940961120494, 2)
MDD 0.21241678614939297
4 0 -5 (3.663253766235682, 4)
MDD 0.2140556614627707
6 0 -5 (3.793165178044753, 6)
MDD 0.211120240550644
8 0 -5 (3.827328421234051, 8)
MDD 0.20941566573683607
10 0 -5 (3.793556517615529, 10)
MDD 0.2143960717230729
12 0 -5 (3.709626638605951, 12)
MDD 0.2178822385168046
14 0 -5 (3.639236537663295, 14)
MDD 0.21625898521193793
16 0 -5 (3.637125549136324, 16)
MDD 0.21630606461575327
18 0 -5 (3.56971249754915, 18)
MDD 0.21336188563139133
2 5 -5 (3.4999146661589675, 2)
MDD 0.2814713755486342
4 5 -5 (2.8479447060109484, 4)
MDD 0.3352863133822443
6 5 -5 (2.701762759650414, 6)
MDD 0.3113357698582291
8 5 -5 (2.3694382168551873, 8)
MDD 0.32280794375355537
10 5 -5 (2.3086500376387074, 10)
MDD 0.34730014695295713
12 5 -5 (2.009135685385712, 12)
MDD 0.3354620741673451
14 5 -5 (1.7933726846418265, 14)
MDD 0.3558851682339103
16 5 -5 (1.9362925019170287, 16)
MDD 0.36519291037807855
18 5 -5 (1.8986196518811558, 18)
MDD 0.26801

In [12]:
optimize_result.sort_values(by=0, ascending =False ).head(50)


,0,1
3,3.827328,8
4,3.793557,10
2,3.793165,6
45,3.768997,2
5,3.709627,12
1,3.663254,4
6,3.639237,14
7,3.637126,16
8,3.569712,18
9,3.499915,2


In [12]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1
3,3.975725,12
2,3.942921,10
4,3.917702,14
5,3.867083,16
6,3.699542,18
7,3.641825,20
1,3.562381,8
8,3.561433,22
9,3.302525,24
10,3.231896,26
